# Data cleaning Immobiliare.it

Importing the librares

In [14]:
import numpy as np
import pandas as pd
import re

I check the raw dataframe and the datatypes

In [15]:
df_raw = pd.read_csv('house_prices_italy.csv')
df_raw.head()

,Unnamed: 0,region,city,area,rooms,toilets,price
0,0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,2,abruzzo,Pescara,227m²,5,3+,€ 299.000
3,3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000
4,4,abruzzo,Rosciano,530m²,5+,3+,€ 650.000


In [16]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  40000 non-null  int64 
 1   region      40000 non-null  object
 2   city        40000 non-null  object
 3   area        39975 non-null  object
 4   rooms       39052 non-null  object
 5   toilets     40000 non-null  object
 6   price       40000 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


Are there any null values? let's see:

In [17]:
df_raw.isnull().sum()

Unnamed: 0      0
region          0
city            0
area           25
rooms         948
toilets         0
price           0
dtype: int64

I decided to drop the first column since it does not carry any info:

In [18]:
df=df_raw.drop("Unnamed: 0", axis=1)

In [19]:
df.head()

,region,city,area,rooms,toilets,price
0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,abruzzo,Pescara,227m²,5,3+,€ 299.000
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000
4,abruzzo,Rosciano,530m²,5+,3+,€ 650.000


I force the "area" column as 'string', if not, later it will give errors.

In [20]:
df['area'] = df['area'].astype(str)

I do not like the column names. I rename them:

In [21]:
df.rename(columns={'area':'area[m2]', 'price':'price[€]'}, inplace=True)
df.sample(10)

,region,city,area[m2],rooms,toilets,price[€]
14861,liguria,Genova,85m²,3,2,€ 270.000
3044,basilicata,Policoro,79m²,4,2,€ 115.000
25058,puglia,Francavilla Fontana,150m²,5+,1,€ 350.000
27878,sardegna,Castiadas,90m²,3,1,€ 270.000
4570,campania,San Cipriano d'Aversa,367m²,5,1,"da € 172.687,00"
33753,trentino-alto-adige,Bolzano,83m²,3,1,€ 310.000
2345,basilicata,Tursi,160m²,4,2,€ 95.000
25812,puglia,Matino,260m²,5+,3,€ 175.000
6053,calabria,Crotone,133m²,3,1,"da € 79.700,00"
33439,trentino-alto-adige,Merano,185m²,3,1,€ 2.000.000


Regex on "area" column. Let's start cleaning from here:

In [22]:
#compilo il pattern
p = re.compile('[0-9]+')


df['area[m2]'] = df['area[m2]'].apply(lambda x: np.nan if p.search(x) is None else p.search(x).group())
df.head()

,region,city,area[m2],rooms,toilets,price[€]
0,abruzzo,Pescara,89,3,1,€ 75.000
1,abruzzo,Spoltore,199,5+,3+,€ 235.000
2,abruzzo,Pescara,227,5,3+,€ 299.000
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43,2 - 4,1,da € 165.000
4,abruzzo,Rosciano,530,5+,3+,€ 650.000


In [23]:
df.isna().sum()

region        0
city          0
area[m2]     25
rooms       948
toilets       0
price[€]      0
dtype: int64

Now it's the turn for the "price" column:

In [24]:
p_price = re.compile('[0-9]+\.[0-9]+')

In [25]:
df['price[€]'] = df['price[€]'].apply(lambda x: np.nan if p_price.search(x) is None else p_price.search(x).group())
df.isna().sum()

region         0
city           0
area[m2]      25
rooms        948
toilets        0
price[€]    1265
dtype: int64

I now rename the regions into a more appropriate manner:

In [26]:
regions_dict = {'abruzzo':'Abruzzo', 'basilicata':'Basilicata', 'campania':'Campania', 'calabria':'Calabria', 'emilia-romagna':'Emilia Romagna',
       'friuli-venezia-giulia':'Friuli-Venezia Giulia', 'lazio': 'Lazio', 'liguria':'Liguria', 'lombardia':'Lombardia', 'marche':'Marche',
       'molise':'Molise', 'piemonte':'Piemonte', 'puglia':'Puglia', 'sardegna':'Sardegna', 'sicilia':'Sicilia', 'toscana':'Toscana',
       'trentino-alto-adige': 'Trentino-Alto Adige', 'umbria':'Umbria', 'valle-d-aosta':'Valle d\'Aosta', 'veneto':'Veneto'}

df.replace({'region':regions_dict}, inplace=True)
df.isna().sum()


region         0
city           0
area[m2]      25
rooms        948
toilets        0
price[€]    1265
dtype: int64

I believe this is enough. It will be possible to load the data into a DataViz software and keep going from there with some visualizations

In [27]:
df.to_csv('house_prices_italy_cleaned.csv')